In [69]:
import requests
import wikipedia
import string
from wikiparse import WikiParser
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

TITLE = "Category:Marvel Comics supervillains"

PARAMS = {
    'action': "query",
    'list': 'categorymembers',
    'cmtitle': TITLE,
    'cmlimit': '100',
    'format': "json",
}

R = S.get(url=URL, params=PARAMS)
DATA = R.json()
ids = [item['title'] for ind,item in enumerate(DATA['query']['categorymembers'])]
ids[:20]

['Abomination (comics)',
 'Absalom (comics)',
 'Absorbing Man',
 'Abyss (comics)',
 'Achilles (Pantheon)',
 'Acrobat (comics)',
 'Adam II',
 'Adria (comics)',
 'Adversary (comics)',
 'Aftershock (comics)',
 'Ahab (comics)',
 'List of incarnations of Spider-Man',
 'Air-Walker',
 'Francis Fanny',
 'Ajaxis',
 'Al-Tariq',
 'Albino (comics)',
 'Alex (comics)',
 'Alkhema',
 'Allatou']

In [5]:
len(ids)

100

In [4]:
bios = []
miss = 0

for id in ids:
    try:
        hero = wikipedia.page(id)
        bio = get_bio(hero)
        if bio != -1:
            bios.append(bio)
    except wikipedia.exceptions.DisambiguationError:
        miss += 1

In [71]:
page = wikipedia.page('abigail Brand')
w = WikiParser()
tests = w.get_all_bios(ids,'villain')
tests[:5]

. . . . . . . . . .  


[{'bio': 'Stan Lee chose the name the Abomination which he realized belonged to no other character before conceiving the characters background and appearance Lee recalled that he simply told the artist Gil Kane to make him bigger and stronger than the Hulk and well have a lot of fun with himEmil Blonsky first appeared in the title Tales to Astonish and was introduced as a KGB agent and spy who became the Abomination after deliberately exposing himself to a greater quantity of the same gamma radiation that transformed Bruce Banner into his alter ego the Hulk using a machine Banner was planning on using to commit suicide In his first appearance Blonsky became a large scaly humanoid even stronger than the Hulk In accordance with Lees wishes the character defeated the Hulk in their first battleThe character has been featured in a number of Marvel titles gradually shifting from unthinking savage brute to master schemer to tortured soul and finally repentant villain and occasional defender o

In [91]:
fulltext = c.content

In [87]:
#S = requests.Session()

wp = WikiParser()
cat = "Category:Marvel Comics superheroes"
titles = wp.get_category(cat)

bios = []
old_bios = []

for _ in range(15):
    print('page: ' + str(_) + ' parsing...', end=" ")
    new_bios = wp.get_all_bios(titles,'hero')
    if new_bios == old_bios:
        break
    bios += new_bios
    if wp.cmc != -1:
        titles = wp.continue_category(cat)
    old_bios = new_bios
    
len(bios)
heroes = bios

page: 0 parsing... . . . . . . . . . .  
page: 1 parsing... . . . . . . . . . .  
page: 2 parsing... . . . . . . . . . .  
page: 3 parsing... . . . . . . . . . .  
page: 4 parsing... . . . . . . . . . .  
page: 5 parsing... . . . . . . . . . .  
page: 6 parsing... . . . . . . . . . .  
page: 7 parsing... . . . . . . . . . .  
page: 8 parsing... . . . . . . . . .  
page: 9 parsing... . . . . . . . . .  


In [85]:
heroes = bios
len(heroes)

378

In [84]:
#S = requests.Session()

wp_v = WikiParser()
cat = "Category:Marvel Comics supervillains"
titles = wp_v.get_category(cat)

bios = []

for _ in range(12):
    print('page: ' + str(_) + ' parsing...', end=" ")
    new_bios = wp_v.get_all_bios(titles,'villain')
    bios += new_bios
    if wp_v.cmc != -1:
        titles = wp_v.continue_category(cat)
    
len(bios)

page: 0 parsing... . . . . . . . . . .  
page: 1 parsing... . . . . . . . . . .  
page: 2 parsing... . . . . . . . . . .  
page: 3 parsing... . . . . . . . . . .  
page: 4 parsing... . . . . . . . . . .  
page: 5 parsing... . . . . . . . . . .  
page: 6 parsing... . . . . . . . . . .  
page: 7 parsing... . . . . . . . . . .  
page: 8 parsing... . . . . . . . . . .  
page: 9 parsing... . . . . . . . . . .  
page: 10 parsing... . . . . . . . . . .  
page: 11 parsing... . .  


378

In [86]:
villains = bios
len(villains)

378

In [55]:
all_marvel = villains + heroes

In [56]:
len(all_marvel)

1399

In [81]:
len(wp_v.ambiguation)

20

In [72]:
import pandas as pd

marvel_df = pd.DataFrame.from_dict(all_marvel)
marvel_df.head()
#marvel_df.to_json('marvel_bios.json')

,bio,label
0,"Stan Lee chose the name ""the Abomination"", whi...",villain
1,The Absorbing Man first appears in Journey int...,villain
2,"=== Creation === In 1940, writer Joe Simon c...",villain
3,None,villain
4,None,villain


In [78]:
marvel_df[~marvel_df['bio'].isna()].shape

(793, 2)

In [66]:
marvel_df.shape

(731, 2)